<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="Logo HEIG-VD" style="width: 80px;" align="right"/>

# Cours APN - Labo 7 : Classification non supervisée

## Résumé
Le but de ce laboratoire est de réaliser une expérience de classification non-supervisée d'articles.  L'approche se base sur des vecteurs en basse dimension (*embeddings*) qui représentent les classes, qui seront comparés avec les vecteurs en basse dimension (*embeddings*) représentant les documents.  Ces *embeddings* seront obtenus soit au modèle `word2vec`.  La méthode sera testée sur un corpus d'articles provenant de rubriques connues, ce qui permettra d'évaluer les méthodes.

In [1]:
# Librairies générales
import pandas as pd
import numpy as np
import random
from tqdm import tqdm

In [2]:
# Similarité entre vecteurs (mots, catégories ou textes)
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# Librairies pour l'évaluation
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, ConfusionMatrixDisplay

## 1. Préparation des données

Vous utiliserez un corpus d'environ 200'000 articles (titres et résumés) [diponibles sur Kaggle](https://www.kaggle.com/datasets/rmisra/news-category-dataset/versions/2) (V2, 80 Mo, nécessite un login) mais dont une copie est **fournie sur Cyberlearn**.  Dans cette partie, vous allez:
  - a. charger le corpus fourni au format JSON, l'explorer et afficher des statistiques
  - b. définir une fonction de normalisation de textes (avec la librairie `utils.py` déjà utilisée)
  - c. définir une fonction d'extraction des textes avec leur catégorie
***

**a.** Veuillez charger le corpus à l'aide des instructions données ci-dessous, puis afficher les statistiques suivantes :
  * un exemple d'article
  * nombre total d'articles
  * nombre d'articles pour chaque catégorie (ou classe) par ordre décroissant
  * nombre d'articles sans `headline`
  * nombre d'articles sans `short_description`
  * nombre d'articles dont la longueur de `headline + short_description` est inférieure ou égale à 2 caractères
  * longueur moyenne de `headline + short_description`

In [4]:
import json
from collections import Counter # pour calculer facilement le nombre d'articles par categorie

file_path = './News_Category_Dataset_v2.json/News_Category_Dataset_v2.json'

with open(file_path, 'r', encoding='utf-8') as file:
    data = [json.loads(line) for line in file]

total_articles = len(data)
print(f"Nombre total d'articles : {total_articles}")


example_article = data[0]
print("\nExemple d'article :", example_article)

categories = [article['category'] for article in data if 'category' in article]
category_counts = Counter(categories)
print("\nNombre d'articles par catégorie")
for category, count in category_counts.most_common():
    print(f"{category}: {count}")

missing_headlines = sum(1 for article in data if not article.get('headline'))
print(f"\nNombre d'articles sans 'headline' : {missing_headlines}")

missing_descriptions = sum(1 for article in data if not article.get('short_description'))
print(f"Nombre d'articles sans 'short_description' : {missing_descriptions}")

short_articles = sum(
    1 for article in data if len(article.get('headline', '') + article.get('short_description', '')) <= 2
)
print(f"Nombre d'articles avec 'headline + short_description' ≤ 2 caractères : {short_articles}")

total_length = sum(
    len(article.get('headline', '') + article.get('short_description', '')) for article in data
)
average_length = total_length / total_articles
print(f"Longueur moyenne de 'headline + short_description' : {average_length:.2f}")



Nombre total d'articles : 200853

Exemple d'article : {'category': 'CRIME', 'headline': 'There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV', 'authors': 'Melissa Jeltsen', 'link': 'https://www.huffingtonpost.com/entry/texas-amanda-painter-mass-shooting_us_5b081ab4e4b0802d69caad89', 'short_description': 'She left her husband. He killed their children. Just another day in America.', 'date': '2018-05-26'}

Nombre d'articles par catégorie
POLITICS: 32739
WELLNESS: 17827
ENTERTAINMENT: 16058
TRAVEL: 9887
STYLE & BEAUTY: 9649
PARENTING: 8677
HEALTHY LIVING: 6694
QUEER VOICES: 6314
FOOD & DRINK: 6226
BUSINESS: 5937
COMEDY: 5175
SPORTS: 4884
BLACK VOICES: 4528
HOME & LIVING: 4195
PARENTS: 3955
THE WORLDPOST: 3664
WEDDINGS: 3651
WOMEN: 3490
IMPACT: 3459
DIVORCE: 3426
CRIME: 3405
MEDIA: 2815
WEIRD NEWS: 2670
GREEN: 2622
WORLDPOST: 2579
RELIGION: 2556
STYLE: 2254
SCIENCE: 2178
WORLD NEWS: 2177
TASTE: 2096
TECH: 2082
MONEY: 1707
ARTS: 1509
FIFTY: 1401
GOOD NEWS: 1398
ARTS & CULTURE: 

In [5]:
data_df = pd.DataFrame(data)

In [6]:
corpus  = []
with open('News_Category_Dataset_v2.json/News_Category_Dataset_v2.json', mode='r', errors='ignore') as json_file:
    for dic in json_file:
        corpus.append(json.loads(dic))

**b.** Veuillez définir une fonction de nettoyage et normalisation des textes, qui vise à réduire la diversité du vocabulaire (lemmatisation, suppression des ponctuations, nombres, ou *stopwords*, etc.).  Veuillez utiliser les fonctions fournies dans la librairie `utils.py` fournie sur Cyberlearn et déjà vue au labo 3 (groupement hiérarchique de films).  Votre fonction devra prendre en entrée un texte non-tokenisé (une chaîne de caractères) et retournera une chaîne de caractères également, mais avec tous les tokens retenus.

In [7]:
import utils

In [8]:
def normalize_text(text):
    texte = utils.remove_punctuation(text)
    texte = utils.remove_non_alphabetic(texte)
    texte = utils.lemmatize_text(texte)
    texte = utils.remove_stopwords(texte)
    
    return texte 

**c.** Veuillez écrire une fonction qui sélectionne les articles d'une ou plusieurs catégories (données sous forme de liste, p.ex. `['MONEY', 'SCIENCE']`) et retourne leurs textes et leurs catégories.  Plus précisément :
* la fonction retourne une liste de textes et une liste de catégories de même longueur (au texte *i* correspond la catégorie ou classe *i*)
* le texte de chaque article est composé de son `headline` et de sa `short_description`, séparés par un point+espace
* si `normalize=True`, la fonction normalise les textes (note : ça ne sera pas toujours souhaitable dans les expériences suivantes)
* on ne retient dans le résultat que les textes dont la longueur finale est supérieure à 3 caractères.

In [9]:
def select_texts_categories(corpus, selected_categories, normalize=True):
    texts = []
    categories = []

    for article in corpus:

        if article.get('category') in selected_categories:
            headline = article.get('headline', '').strip()
            short_description = article.get('short_description', '').strip()
            text = f"{headline}. {short_description}" if headline or short_description else ""
            
            if normalize and text:
                text = normalize_text(text)
            
            if len(text) > 3:
                texts.append(text)
                categories.append(article['category'])
    
    return texts, categories


**d.** Veuillez exécuter la fonction, en appliquant la normalisation des textes, puis afficher un exemple de résultat et commenter brièvement son contenu.  Le code est donné ci-dessous, et vous devez ajouter votre commentaire.

In [10]:
selected_categories = ['MONEY', 'SCIENCE']
texts, categories = select_texts_categories(corpus, selected_categories, normalize=True)

In [11]:
print("Exemple :", categories[142], ':', texts[142])

Exemple : SCIENCE : fireball rip siberian sky brilliant display light frankly scar think bomb witness


In [12]:
# Votre commentaire ici.
# Nous pouvons voir que le texte normalisé est exempt de "" ou autres signes de ponctuations.
# Nous remarquons que les mots sémantiquement importants sont conservés comme dans l'exemple ci-dessus.
# La lemmatisation est également effectuées puisque nous voyons des mots comme "scar". 
# 


___
## 2. Classification non supervisée avec des embeddings word2vec

La méthode de classification proposée comporte trois étapes.  Le but de cette partie est de définir une fonction pour chacune d'elles.  Au début, une (re)prise en main de word2vec est demandée.
* a. prise en main de word2vec
* b. création des représentations vectorielles (*embeddings*) des classes (catégories)
* c. création des représentations vectorielles (*embeddings*) d'un texte
* d. classification : comparer les similarités du vecteur de texte avec les vecteurs des classes, choisir la plus similaire
___

**a.** Prise en main de word2vec.  Vous avez déjà utilisé word2vec au Labo 4 sur la visualisation de vecteurs de mots (et Cémantix) mais ici vous utiliserez un modèle pour l'anglais.  Vous pouvez consulter la [documentation de gensim sur KeyedVectors](https://radimrehurek.com/gensim/models/keyedvectors.html#what-can-i-do-with-word-vectors)). 

In [13]:
from gensim.models import KeyedVectors
import gensim.downloader

In [14]:
# Si le modèle n'est pas téléchargé :
#w2v_model = gensim.downloader.load('word2vec-google-news-300')
# S'il l'est déjà, indiquer son emplacement :
path_to_model = "C:\\Users\\Julien\\gensim-data\\word2vec-google-news-300\\word2vec-google-news-300.gz"
w2v_model = KeyedVectors.load_word2vec_format(path_to_model, binary=True, unicode_errors="ignore")
# Attention, ce modèle prend environ 5 Go en mémoire. 

Veuillez afficher les mots les plus similaires selon ce modèle word2vec du mot 'science'.  Même question pour le mot 'money' (ce sont des noms de catégories).  Que pensez-vous du résultat ?  

In [15]:
similar_to_science = w2v_model.most_similar("science", topn=10)
print("Mots similaires à 'science' :")
for word, similarity in similar_to_science:
    print(f"{word}: {similarity:.4f}")

similar_to_money = w2v_model.most_similar("money", topn=10)
print("\nMots similaires à 'money' :")
for word, similarity in similar_to_money:
    print(f"{word}: {similarity:.4f}")

Mots similaires à 'science' :
faith_Jezierski: 0.6965
sciences: 0.6821
biology: 0.6776
scientific: 0.6535
mathematics: 0.6301
Hilal_Khashan_professor: 0.6153
impeach_USADA: 0.6149
professor_Kent_Redfield: 0.6144
physics_astronomy: 0.6105
bionic_prosthetic_fingers: 0.6083

Mots similaires à 'money' :
monies: 0.7165
funds: 0.7055
moneys: 0.6289
dollars: 0.6289
cash: 0.6151
vast_sums: 0.6057
fund: 0.5790
Money: 0.5733
taxpayer_dollars: 0.5694
Monies: 0.5587


Basé sur les résultats précédents et vos intuitions sur les articles qu'on peut trouver dans les catégories `['SCIENCE', 'MONEY']`, veuillez indiquer ici 5 à 10 mots représentatifs de chacune de ces deux catégories.

In [16]:
data_df[ data_df['category'].isin(['MONEY', 'SCIENCE']) ]['short_description']
#
# Nous pouvons citer les mots similaires à 'science' : biology, sciences, scientific, chemistry, physics, neuroscience, mathematics, biochemistry, biophysics, et biotechnology.

# Nous pouvons citer les mots similaires à 'money' : cash, dollars, money, funds, currency, euros, million, billions, and trillions.
#

155       The researchers plan to scour the Loch Ness ne...
285       The supposed "interstellar immigrant" is locat...
439       It's the first time a rocket designed by a Chi...
449                                                  YIKES!
1246      Some of America's top researchers will move to...
                                ...                        
200754    Because of the overuse of antibiotics, antibio...
200815    Gallery: Space Station's Expedition 30 Mission...
200816    image 1: throw As Hizook reports, DLR started ...
200817    That doesn't mean Jobs lacks for fans in the w...
200818    Aurora borealis can typically only be seen at ...
Name: short_description, Length: 3885, dtype: object

**b.** Création des représentations vectorielles des classes (catégories).

Veuillez définir une fonction qui retourne un vecteur (*embedding*) word2vec pour chacune des classes d'un tableau qui est fourni en argument de la fonction.  Consignes :

* la fonction retourne la moyenne des *embeddings* des mots-clés associés à chaque classe;
* pour les classes 'SCIENCE' et 'MONEY', elle prend les mots-clés que vous avez choisis ci-dessus (au (a)) ;
* pour les autres classes, elle demande à word2vec les `topn` voisins du nom de la classe (en minuscules) ;
  - si `topn = 0`, on utilise seulement le nom de la classe (en minuscules) ;
  - on suppose que le nom de la classe est connu de word2vec.


In [17]:
import numpy as np
from gensim.models import KeyedVectors

def cat_embedding_w2v(model, category_names, topn=30):

    category_embeddings = {}
    
    # Mots-clés pour SCIENCE et MONEY
    category_keywords = {
        'SCIENCE': ["research", "technology", "experiment", "discovery", "biology", "physics", "chemistry", "innovation", "astronomy", "data"],
        'MONEY': ["finance", "investment", "market", "currency", "stock", "wealth", "cash", "banking", "economy", "business"]
    }
    
    for category in category_names:

        if category in category_keywords:
            words = category_keywords[category]
        else:
            # Sinon, récupérer les voisins du nom de la catégorie
            words = [category.lower()]
            if topn > 0:
                try:
                    # Récupérer les mots les plus proches du nom de la catégorie
                    similar_words = model.most_similar(category.lower(), topn=topn)
                    words.extend([word for word, _ in similar_words])
                except KeyError:
                    print(f"Le mot '{category.lower()}' n'a pas été trouvé dans le vocabulaire du modèle.")

        # Calculer la moyenne des embeddings des mots
        embeddings = []
        for word in words:
            try:
                embeddings.append(model[word])
            except KeyError:
                pass
        # Calculer la moyenne des embeddings
        if embeddings:
            category_embeddings[category] = np.mean(embeddings, axis=0)
        else:
            category_embeddings[category] = np.zeros(model.vector_size)
    return category_embeddings


In [18]:
# Test : affiche-t-on 'True' ?
# faire un array depuis le dictionnaire
e1 = np.array(cat_embedding_w2v(w2v_model, ['SCIENCE'], topn=30)['SCIENCE'])
e2 = np.array(cat_embedding_w2v(w2v_model, ['TECH'], topn=30)['TECH'])
e3 = np.array(cat_embedding_w2v(w2v_model, ['TASTE'], topn=30)['TASTE'])

e1 = e1.reshape(1, -1)
e2 = e2.reshape(1, -1)
e3 = e3.reshape(1, -1)

result = cosine_similarity(e1, e2) > cosine_similarity(e2, e3)
print(result)

[[ True]]


**c.** Création de la représentation vectorielle d'un texte

Veuillez définir une fonction qui prend un texte (*string*) en argument et retourne un vecteur (*embedding*) qui représente le texte.  Le texte doit être découpé en mots (tokenisé), puis on doit tester si chaque mot est connu du modèle word2vec, et si oui, on récupère le *embedding* du mot.  La fonction retourne la moyenne des *embeddings*, sauf si aucun mot du texte n'est connu du modèle word2vec, auquel cas elle retourne `[]`.

In [19]:
from nltk import word_tokenize

In [20]:
def text_embedding_w2v(model, text):
    words = word_tokenize(text.lower())
    embeddings = []
    
    for word in words:
        try:
            # Tente d'accéder à l'embedding du mot
            embeddings.append(model[word])
        except KeyError:
            # Si le mot n'est pas trouvé dans le vocabulaire du modèle, on l'ignore
            continue
    
    # Si au moins un mot connu est trouvé, on retourne la moyenne des embeddings
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return []  

**d.** Classification non supervisée d'articles avec word2vec

Veuillez définir une fonction qui prend en entrée :
* un modèle word2vec
* une liste de textes à classifier
* une liste de catégories définies par leur nom en majuscules (p.ex. `['SCIENCE', 'MONEY']`)
et qui retourne le tableau avec la catégorie prédite pour chaque texte parmi les catégories données.  

Pour prédire la catégorie, la fonction calcule la similarité cosinus du *embedding* du texte avec chacun des *embeddings* des catégories, et choisit la catégorie qui a la plus grande similarité.  Si le texte n'a pas de *embedding* (parce qu'aucun de ses mots n'est connu du modèle), ou si plusieurs catégories ont la même similarité, on tire au sort la catégorie.

In [21]:
import numpy as np
from scipy.spatial.distance import cosine
import random

def classify_w2v(model, texts, selected_categories, topn = 30):

    category_embeddings = cat_embedding_w2v(model, selected_categories, topn)
    cat_pred = []

    for text in texts:
        text_embedding = text_embedding_w2v(model, text)
        
        if len(text_embedding) == 0:
            # Si aucun embedding n'est trouvé, tirer une catégorie au sort
            predicted_category = random.choice(selected_categories)
        else:
            # Calculer la similarité cosinus entre le texte et chaque catégorie
            similarities = {}
            for category, category_embedding in category_embeddings.items():
                if np.linalg.norm(category_embedding) != 0:
                    similarity = 1 - cosine(text_embedding, category_embedding)
                else:
                    similarity = -1
                similarities[category] = similarity
            
            max_similarity = max(similarities.values())
            best_categories = [cat for cat, sim in similarities.items() if sim == max_similarity]
            
            predicted_category = random.choice(best_categories)
        
        cat_pred.append(predicted_category)
    
    return cat_pred

**e.** Veuillez réaliser la classification non supervisée des articles des catégories `['SCIENCE', 'MONEY']`.  Afficher les scores obtenus et la matrice de confusion en utilisant les fonctions de `sklearn.metrics` importées au début du notebook.  Veuillez faire plusieurs essais pour optimiser les fonctions et leurs appels, et à la fin laisser votre meilleur résultat dans ce notebook.  

In [22]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

selected_categories = ['SCIENCE', 'MONEY']
texts, categories = select_texts_categories(corpus, selected_categories, normalize=True)

predicted_categories = classify_w2v(w2v_model, texts, categories, topn=30)
predicted_categories_filtered = predicted_categories[:len(categories)]

accuracy = accuracy_score(categories, predicted_categories_filtered)
conf_matrix = confusion_matrix(categories, predicted_categories_filtered, labels=selected_categories)
class_report = classification_report(categories, predicted_categories_filtered, labels=selected_categories)

print(f"Exactitude : {accuracy}")
print(f"Matrice de confusion :\n{conf_matrix}")
print(f"Rapport de classification :\n{class_report}")


Exactitude : 0.9297297297297298
Matrice de confusion :
[[2026  152]
 [ 121 1586]]
Rapport de classification :
              precision    recall  f1-score   support

     SCIENCE       0.94      0.93      0.94      2178
       MONEY       0.91      0.93      0.92      1707

    accuracy                           0.93      3885
   macro avg       0.93      0.93      0.93      3885
weighted avg       0.93      0.93      0.93      3885



**f.** Veuillez réaliser une expérience de classification non supervisée sur les articles des catégories `['TECH', 'ARTS', 'COLLEGE']` en variant le paramètre `topn` de `classify_w2v` et en indiquant la meilleure valeur trouvée.

In [23]:
len(categories)

3885

In [24]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

selected_categories = ['TECH', 'ARTS', 'COLLEGE']
texts, categories = select_texts_categories(corpus, selected_categories, normalize=False)

topn_values = [10, 20, 30]
best_topn = None
best_accuracy = 0
best_confusion_matrix = None

for topn in topn_values:
    predicted_categories = classify_w2v(w2v_model, texts, selected_categories, topn=topn)
    predicted_categories_filtered = predicted_categories[:len(categories)]

    accuracy = accuracy_score(categories, predicted_categories_filtered)
    conf_matrix = confusion_matrix(categories, predicted_categories_filtered, labels=selected_categories)

    print(f"Topn: {topn}")
    print(f"Accuracy: {accuracy}")
    print(f"Confusion Matrix:\n{conf_matrix}")

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_topn = topn
        best_confusion_matrix = conf_matrix

print(f"\nMeilleur topn: {best_topn}")
print(f"Meilleure précision: {best_accuracy}")
print(f"Matrice de confusion pour topn = {best_topn}:\n{best_confusion_matrix}")

predicted_categories_final = classify_w2v(w2v_model, texts, selected_categories, topn=best_topn)
predicted_categories_filtered_final = predicted_categories_final[:len(categories)]

class_report = classification_report(categories, predicted_categories_filtered_final, labels=selected_categories)
print(f"\nRapport de classification pour topn = {best_topn}:\n{class_report}")


Topn: 10
Accuracy: 0.7852164730728617
Confusion Matrix:
[[1738   79  265]
 [ 262  983  264]
 [  79   68  997]]
Topn: 20
Accuracy: 0.8101372756071805
Confusion Matrix:
[[1880   77  125]
 [ 334 1027  148]
 [ 140   75  929]]
Topn: 30
Accuracy: 0.7788806758183738
Confusion Matrix:
[[1811   39  232]
 [ 365  853  291]
 [  90   30 1024]]

Meilleur topn: 20
Meilleure précision: 0.8101372756071805
Matrice de confusion pour topn = 20:
[[1880   77  125]
 [ 334 1027  148]
 [ 140   75  929]]

Rapport de classification pour topn = 20:
              precision    recall  f1-score   support

        TECH       0.80      0.90      0.85      2082
        ARTS       0.87      0.68      0.76      1509
     COLLEGE       0.77      0.81      0.79      1144

    accuracy                           0.81      4735
   macro avg       0.81      0.80      0.80      4735
weighted avg       0.82      0.81      0.81      4735



**g.** Veuillez comparer les deux expériences (points (e) et (f)) en termes de scores, de valeurs optimales de `topn`, et de l'impact du nettoyage de textes.   Veuillez donner votre opinion sur la qualité de la classification non supervisée avec word2vec.

**Fin du Labo.**  Veuillez nettoyer ce notebook en gardant seulement les résultats et les commentaires demandés, l'enregistrer, et le soumettre comme devoir sur Cyberlearn.